# 🎬 Analyse IMDb - Préparation des données

**Objectif** : Créer un dataset propre avec films, notes, genres et acteurs/réalisateurs

In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import os

# Configuration
pd.set_option('display.max_columns', None)

## 📁 Configuration des chemins (relatifs pour GitHub)

In [ ]:
# Chemins relatifs (compatible GitHub)
PROJECT_ROOT = Path.cwd().parent if 'notebooks' in str(Path.cwd()) else Path.cwd()
DATA_DIR = PROJECT_ROOT / "data"
PARQUET_DIR = DATA_DIR / "PARQUETS"

# Créer les dossiers
PARQUET_DIR.mkdir(parents=True, exist_ok=True)
print(f"✅ Dossier créé : {PARQUET_DIR}")

## 📥 ÉTAPE 1 : Charger et filtrer title.basics

In [ ]:
# URL IMDb
url_basics = "https://datasets.imdbws.com/title.basics.tsv.gz"

# Chemin de sauvegarde
parquet_basics = PARQUET_DIR / "title_basics_movies.parquet"

# Paramètres
chunk_size = 500_000
colonnes_utiles = [
    'tconst', 'titleType', 'primaryTitle', 'originalTitle',
    'isAdult', 'startYear', 'runtimeMinutes', 'genres'
]

# Lecture par chunks
chunks_filtres = []
print("🔄 Traitement de title.basics...")

for i, chunk in enumerate(pd.read_csv(
    url_basics,
    sep='\t',
    chunksize=chunk_size,
    usecols=colonnes_utiles,
    dtype={'startYear': str, 'runtimeMinutes': str, 'isAdult': str},
    na_values='\\N'
), 1):
    
    # Convertir isAdult en numérique (0 ou 1)
    chunk['isAdult'] = pd.to_numeric(chunk['isAdult'], errors='coerce').fillna(1).astype(int)
    
    # ✅ FILTRES COMPLETS
    chunk_filtre = chunk[
        (chunk['titleType'] == 'movie') &
        (chunk['isAdult'] == 0) &  # ← IMPORTANT : Exclure films adultes (0 = non-adulte)
        (chunk['startYear'].notna())
    ].copy()
    
    # Conversions
    chunk_filtre['startYear'] = pd.to_numeric(chunk_filtre['startYear'], errors='coerce')
    chunk_filtre['runtimeMinutes'] = pd.to_numeric(chunk_filtre['runtimeMinutes'], errors='coerce')
    
    # Filtre sur les années (après 1970)
    chunk_filtre = chunk_filtre[chunk_filtre['startYear'] > 1970]
    
    chunks_filtres.append(chunk_filtre)
    print(f"  Chunk {i} : {len(chunk_filtre):,} films gardés")

# Concaténation
df_movies = pd.concat(chunks_filtres, ignore_index=True)
print(f"\n✅ Total : {len(df_movies):,} films conservés")
print(f"📊 Période : {df_movies['startYear'].min():.0f} - {df_movies['startYear'].max():.0f}")

# Sauvegarde
df_movies.to_parquet(parquet_basics, engine='pyarrow', compression='snappy', index=False)
print(f"💾 Sauvegardé : {parquet_basics}")

## 📥 ÉTAPE 2 : Charger title.ratings

In [ ]:
# URL IMDb
url_ratings = "https://datasets.imdbws.com/title.ratings.tsv.gz"
parquet_ratings = PARQUET_DIR / "title_ratings.parquet"

print("🔄 Traitement de title.ratings...")

# Lecture
df_ratings = pd.read_csv(url_ratings, sep='\t', na_values='\\N')

# Filtrer pour garder seulement nos films
tconst_movies = set(df_movies['tconst'])
df_ratings = df_ratings[df_ratings['tconst'].isin(tconst_movies)]

print(f"✅ {len(df_ratings):,} ratings conservés")

# Sauvegarde
df_ratings.to_parquet(parquet_ratings, engine='pyarrow', compression='snappy', index=False)
print(f"💾 Sauvegardé : {parquet_ratings}")

## 🇫🇷 ÉTAPE 3 : Charger les titres régionaux (France)

In [ ]:
# URL IMDb
url_akas = "https://datasets.imdbws.com/title.akas.tsv.gz"
parquet_akas = PARQUET_DIR / "title_akas_fr.parquet"

print("🔄 Traitement de title.akas (titres régionaux)...")

# Paramètres
chunks_filtres = []
chunk_size = 500_000

for i, chunk in enumerate(pd.read_csv(
    url_akas,
    sep='\t',
    chunksize=chunk_size,
    usecols=['titleId', 'title', 'region', 'language', 'types', 'isOriginalTitle'],
    na_values='\\N'
), 1):
    
    # Filtrer uniquement les films qu'on a + région FR
    chunk_filtre = chunk[
        (chunk['titleId'].isin(tconst_movies)) &
        (chunk['region'] == 'FR')
    ].copy()
    
    chunks_filtres.append(chunk_filtre)
    
    if i % 10 == 0:
        print(f"  {i} chunks traités")

df_akas = pd.concat(chunks_filtres, ignore_index=True)
print(f"\n✅ {len(df_akas):,} titres français trouvés")

# Renommer pour clarté
df_akas = df_akas.rename(columns={'titleId': 'tconst', 'title': 'frenchTitle'})

# Garder le meilleur titre FR par film (priorité au titre original si dispo en FR)
df_akas['priority'] = df_akas['isOriginalTitle'].fillna(0).astype(int)
df_akas = df_akas.sort_values('priority', ascending=False)
df_akas = df_akas.drop_duplicates(subset='tconst', keep='first')
df_akas = df_akas[['tconst', 'frenchTitle', 'region']]

print(f"📊 {len(df_akas):,} films avec titre français unique")

# Sauvegarde
df_akas.to_parquet(parquet_akas, engine='pyarrow', compression='snappy', index=False)
print(f"💾 Sauvegardé : {parquet_akas}")

## 🔗 ÉTAPE 4 : Créer le dataset complet avec genres encodés

In [ ]:
# Jointure movies + ratings + titres français
print("🔗 Jointure movies + ratings + titres français...")
df_complet = df_movies.merge(df_ratings, on='tconst', how='left')
df_complet = df_complet.merge(df_akas, on='tconst', how='left')

print(f"✅ Dataset complet : {len(df_complet):,} films")
print(f"📊 Films avec notes : {df_complet['averageRating'].notna().sum():,}")
print(f"🇫🇷 Films avec titre français : {df_complet['frenchTitle'].notna().sum():,}")

# Encoder les genres
print("\n🎭 Encodage des genres...")
df_complet['genres'] = df_complet['genres'].fillna('')
df_complet = pd.concat([
    df_complet,
    df_complet['genres'].str.get_dummies(sep=',')
], axis=1)

genre_cols = [col for col in df_complet.columns if col not in df_movies.columns and col not in df_ratings.columns and col not in df_akas.columns]
print(f"✅ {len(genre_cols)} genres encodés")

# Aperçu
print("\n🔍 Aperçu :")
display(df_complet[['primaryTitle', 'frenchTitle', 'startYear', 'genres', 'averageRating']].head())

# Sauvegarde intermédiaire
parquet_complet = PARQUET_DIR / "imdb_complet_base.parquet"
df_complet.to_parquet(parquet_complet, engine='pyarrow', compression='snappy', index=False)
print(f"💾 Sauvegardé : {parquet_complet}")

## 🎭 ÉTAPE 5 : Ajouter les acteurs et réalisateurs

In [ ]:
# URL IMDb
url_principals = "https://datasets.imdbws.com/title.principals.tsv.gz"

# Paramètres
chunks_filtres = []
chunk_size = 500_000

print("🔄 Traitement de title.principals (peut prendre 10 min)...")

for i, chunk in enumerate(pd.read_csv(
    url_principals,
    sep='\t',
    chunksize=chunk_size,
    usecols=['tconst', 'ordering', 'nconst', 'category'],
    na_values='\\N'
), 1):
    
    # Filtre : Top 5 acteurs + tous les réalisateurs
    chunk_filtre = chunk[
        (chunk['tconst'].isin(tconst_movies)) &
        (
            ((chunk['category'].isin(['actor', 'actress'])) & (chunk['ordering'] <= 5)) |
            (chunk['category'] == 'director')
        )
    ].copy()
    
    chunks_filtres.append(chunk_filtre)
    
    if i % 10 == 0:
        print(f"  {i} chunks traités")

df_principals = pd.concat(chunks_filtres, ignore_index=True)
print(f"\n✅ {len(df_principals):,} lignes conservées")

# Sauvegarder
parquet_principals = PARQUET_DIR / "title_principals_filtered.parquet"
df_principals.to_parquet(parquet_principals, engine='pyarrow', compression='snappy', index=False)
print(f"💾 Sauvegardé : {parquet_principals}")

In [ ]:
# ⚡ CHARGEMENT OPTIMISÉ de name.basics
# On charge SEULEMENT les personnes présentes dans nos films (50-100K au lieu de 13M+)

url_names = "https://datasets.imdbws.com/name.basics.tsv.gz"

# Extraire la liste des nconst nécessaires depuis df_principals
nconst_needed = set(df_principals['nconst'].unique())
print(f"🎯 Personnes à rechercher : {len(nconst_needed):,}")

print("\n🔄 Chargement OPTIMISÉ de name.basics (filtrage immédiat)...")
print("   Cette étape peut prendre 3-5 minutes...")

chunks_names = []
chunk_size = 500_000
total_found = 0

for i, chunk in enumerate(pd.read_csv(
    url_names,
    sep='\t',
    chunksize=chunk_size,
    usecols=['nconst', 'primaryName'],
    na_values='\\N'
), 1):
    # ⚡ FILTRAGE IMMÉDIAT : Ne garder QUE les personnes nécessaires
    chunk_filtre = chunk[chunk['nconst'].isin(nconst_needed)]
    
    if len(chunk_filtre) > 0:
        chunks_names.append(chunk_filtre)
        total_found += len(chunk_filtre)
    
    # Affichage progression tous les 10 chunks
    if i % 10 == 0:
        print(f"  {i} chunks traités | {total_found:,} noms trouvés")
    
    # ⚡ OPTIMISATION : Arrêter si on a trouvé tout le monde
    if total_found >= len(nconst_needed):
        print(f"  ✅ Tous les noms trouvés après {i} chunks !")
        break

# Concaténation
df_names = pd.concat(chunks_names, ignore_index=True)

# Supprimer les doublons (au cas où)
df_names = df_names.drop_duplicates(subset='nconst')

print(f"\n✅ {len(df_names):,} noms récupérés sur {len(nconst_needed):,} recherchés")
print(f"📊 Taux de couverture : {len(df_names) / len(nconst_needed) * 100:.1f}%")

# Sauvegarder (optionnel mais utile pour débug)
parquet_names = PARQUET_DIR / "name_basics_filtered.parquet"
df_names.to_parquet(parquet_names, engine='pyarrow', compression='snappy', index=False)
print(f"💾 Sauvegardé : {parquet_names}")

In [ ]:
# ✅ FUSION avec les noms complets

print("🔗 Jointure principals + names...")
df_principals_noms = df_principals.merge(df_names, on='nconst', how='left')

# Créer les listes d'acteurs par film (avec NOMS COMPLETS)
acteurs = df_principals_noms[
    df_principals_noms['category'].isin(['actor', 'actress'])
].groupby('tconst')['primaryName'].apply(list).reset_index(name='acteurs')

# Créer les listes de réalisateurs par film (avec NOMS COMPLETS)
realisateurs = df_principals_noms[
    df_principals_noms['category'] == 'director'
].groupby('tconst')['primaryName'].apply(list).reset_index(name='realisateurs')

print(f"✅ {len(acteurs):,} films avec acteurs")
print(f"✅ {len(realisateurs):,} films avec réalisateurs")

# Joindre à df_complet
print("\n🔗 Ajout du casting au dataset principal...")
df_complet = df_complet.merge(acteurs, on='tconst', how='left')
df_complet = df_complet.merge(realisateurs, on='tconst', how='left')

# Remplacer NaN par listes vides
df_complet['acteurs'] = df_complet['acteurs'].apply(lambda x: x if isinstance(x, list) else [])
df_complet['realisateurs'] = df_complet['realisateurs'].apply(lambda x: x if isinstance(x, list) else [])

print(f"\n✅ Dataset final : {df_complet.shape}")
print(f"\n🎬 Exemples de casting :")

# Afficher quelques exemples
for idx in range(min(3, len(df_complet))):
    row = df_complet.iloc[idx]
    if len(row['acteurs']) > 0 or len(row['realisateurs']) > 0:
        print(f"\n  📽️ {row['primaryTitle']} ({int(row['startYear']) if pd.notna(row['startYear']) else '?'})")
        if len(row['acteurs']) > 0:
            print(f"     🎭 Acteurs : {', '.join(row['acteurs'][:3])}")
        if len(row['realisateurs']) > 0:
            print(f"     🎬 Réalisateur(s) : {', '.join(row['realisateurs'])}")

## 💾 ÉTAPE 6 : Sauvegarde finale

In [ ]:
# Sauvegarde finale
parquet_final = PARQUET_DIR / "imdb_complet_avec_cast.parquet"
df_complet.to_parquet(parquet_final, engine='pyarrow', compression='snappy', index=False)

print("="*60)
print("✅ TRAITEMENT TERMINÉ !")
print("="*60)
print(f"\n💾 Dataset final : {parquet_final}")
print(f"📊 Dimensions : {df_complet.shape}")
print(f"\n📋 Statistiques :")
print(f"  • Films : {len(df_complet):,}")
print(f"  • Avec notes : {df_complet['averageRating'].notna().sum():,}")
print(f"  • Avec titres français : {df_complet['frenchTitle'].notna().sum():,}")
print(f"  • Avec acteurs : {df_complet['acteurs'].apply(len).gt(0).sum():,}")
print(f"  • Avec réalisateurs : {df_complet['realisateurs'].apply(len).gt(0).sum():,}")

print("\n🔍 Aperçu final :")
display(df_complet[['primaryTitle', 'frenchTitle', 'startYear', 'genres', 'averageRating', 'acteurs', 'realisateurs']].head())